In [1]:
### --- imports ---
import math
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
# Constants (Tjek onenote)
lambda_g = 0.150608 # Guided wavelength of antenna
e_ff = 3.74         # Effective dielectric constant
d = 0.0241          # 2.41 cm
s = 0.0633          # 6.33 cm
L = 0.1054          # 10.54 cm
w = 0.0075          # 0.75 cm

In [58]:
#   Class for Meander line coordinates
#   Can return r_hat vectors for each dipole orientation

class CoordinateSet:
    def __init__(self, coords, lengths):
        self.coords = np.array(coords, dtype=float) # shape (N,3) -----> Maybe add the length so we can calc based on length of each side (some are smaller)
        self.lengths = np.array(lengths, dtype=float)

    def as_array(self):
        #Return the (N,3) numpy array of (x,y,z).
        return self.coords
    
    def lengths_array(self):
        # Return the lengths
        return self.lengths
    
    def r_vectors(self):
        #Return full 3D vectors (x,y,z) as (N,3).
        return self.coords
    
    def r_norms(self):
        # Return distances from origin (N,).
        return np.linalg.norm(self.r_vectors(), axis=1)
    
    def r_hat(self):
        # Return unit vectors pointing from origin to each (x,y,z)
        r_vecs = self.r_vectors()
        return r_vecs / self.r_norms()[:,None]
    
coords = [[0,-1,0], [1,0,0], [0,-1,0], [-1,0,0], [0,-1,0], [0,1,0], [0,-1,0], [-1,0,0], [-1,0,0], [1,0,0]]    # x-axis, y-axis, z-axis
lengths = [0.5*d, s, d, s, d, s, d, s, d, 0.5*d]            # lengths

meander = CoordinateSet(coords, lengths)

print(meander.as_array())
print(meander.lengths_array())


[[ 0. -1.  0.]
 [ 1.  0.  0.]
 [ 0. -1.  0.]
 [-1.  0.  0.]
 [ 0. -1.  0.]
 [ 0.  1.  0.]
 [ 0. -1.  0.]
 [-1.  0.  0.]
 [-1.  0.  0.]
 [ 1.  0.  0.]]
[0.01205 0.0633  0.0241  0.0633  0.0241  0.0633  0.0241  0.0633  0.0241
 0.01205]


Assuming a uniform microstrip with per-unit parameters collapsed into: <br>
    $Z_0 \\$
    $\gamma = \alpha + j\beta$

The current along a lossy matched transmission line is a single forward traveling wave: <br>
$I(\ell) = I_0 e^{-j\gamma \ell}$, <br>
where $I_0$ is the current at the input terminals and $\ell$ is the length of the transmission (meander) line

In [3]:
#   Current distribution along transmission line
#   Derived from transmission line theory
#   Matched load assumed → forward-traveling wave only (no reflections)

# Lossy transmission line
def currentDistLossy(I_0, l, alpha, beta):
    return I_0 * np.exp(-(alpha + 1j*beta)*l)

# Lossless transmission line
def currentDistLossless(I_0, l, alpha, beta):
    return I_0 * np.exp(-1j*beta*l)

The E-field from an infinitesimal dipole antenna:  

$$
E_{r} = \eta \frac{I_0 l \cos(\theta)}{2\pi r^2} \left[ 1 + \frac{1}{jkr} \right] e^{-jkr}
$$

$$
E_{\theta} = j \eta \frac{k I_0 l \sin(\theta)}{4 \pi r} \left[ 1 + \frac{1}{jkr} - \frac{1}{(kr)^2} \right] e^{-jkr}
$$

$$
E_{\phi} = 0
$$

We know:

$$
\underline{\hat{r}} = \underline{\hat{s}}
$$

$$
\underline{\hat{\phi}} = \frac{\underline{\hat{h}} \times \underline{\hat{s}}}{\lVert \underline{\hat{h}} \times \underline{\hat{s}} \rVert}
$$

$$
\underline{\hat{\theta}} = \frac{(\underline{\hat{s}} \cdot \underline{\hat{h}}) \underline{\hat{s}} - \underline{\hat{h}}}{\lVert (\underline{\hat{s}} \cdot \underline{\hat{h}}) \underline{\hat{s}} - \underline{\hat{h}} \rVert}
$$

Also:

$$
R' = \lVert \underline{s} \rVert
$$

$$
\lVert \underline{\hat{h}} \times \underline{\hat{s}} \rVert = \sin \theta
$$

$$
\underline{\hat{s}} \cdot \underline{\hat{h}} = \cos \theta
$$

And:

$$
\underline{V}_{r\theta\phi} =
\begin{bmatrix}
V_r \\
V_\theta \\
V_\phi
\end{bmatrix}
=
\begin{bmatrix}
\sin\theta \cos\phi & \sin\theta \sin\phi & \cos\theta \\
\cos\theta \cos\phi & \cos\theta \sin\phi & -\sin\theta \\
-\sin\phi & \cos\phi & 0
\end{bmatrix}
\begin{bmatrix}
V_x \\
V_y \\
V_z
\end{bmatrix}
$$

where $\hat{h}$ = dipole axis unit vector, $\hat{s}$ = normalized direction vector from the Hertz dipole to the observer point.


In [57]:
# --- Get field from single dipole with length l at a single observation point R---
def singleDipole(I0, l, h_hat, P):
    """
    Needs inputs:
    I0:     Constant
    l:      Length of infinitesimal dipole
    h_hat:  Dipole axis normal vector [defined in meander.r_hat]
    P:      Observation point
    """

    # Define constants
    c0 = 299_792_458    # Speed of light
    eta = 120*np.pi     # Free-space impedance
    freq = 1030e6       # Frequency from paper
    Lambda = c0/freq    # Free-space wavelength 
    k = 2*np.pi/Lambda  # Wave number

    P = np.asarray(P, dtype=float)          # Make sure Point is an array
    r = np.linalg.norm(P)                   # Length from origin to point 
    s_hat = P / r                           # Unit vector of r = s_hat
    h_hat = h_hat / np.linalg.norm(h_hat)   # Unit vector of dipole axis

    rh = s_hat                                                              # r_hat
    ph = (np.cross(h_hat,s_hat)) / (np.linalg.norm(np.cross(h_hat,s_hat)))  # phi_hat
    th = np.cross(ph, rh)                                                   # theta_hat

    # Calculate radial component of E-field
    E_r0 = (eta*I0*l*(np.dot(s_hat,h_hat))) / (2*np.pi*r**2)
    E_r1 = (1 + 1/(1j*k*r)) * np.exp(-1j*k*r)
    E_r = E_r0 * E_r1

    # Calculate theta component of E-field
    E_t0 = (1j*eta*k*I0*l * np.linalg.norm(np.cross(h_hat, s_hat))) * np.exp(-1j*k*r) / (4*np.pi*r)
    E_t1 = (1 + 1/(1j*k*r) - 1/((k*r)**2))
    E_t = E_t0 * E_t1

    E_p = 0

    return E_r, E_t, E_p

In [ ]:
def singleDipoleAllPoints():
    """
    Calculate the electric field at all points from single hertzian antenna

    - Define radius --> maybe normalize?
    - Iterate singleDipole() over all points P
    """
    
    
    pass

In [ ]:
def allDipolesAllPoints():
    """
    Calculate the electric field at all points from all hertzian antennas

    - Iterate singleDipoleAllPoints() for all dipole segments
    """
    
    
    pass